# Classification using Logistic Regression:

 - If you use _linear regression_ in a classification setting, the predicted y will be a continuous variables and not guaranteed to be between 0 and 1
 - Since we want to ensure that the predicted y is in between 0 and 1 to represent probability of "has_covid", we will use _logistic regression_
 - Further reading: [Difference between linear regression and logistic classifier](https://www.analyticsvidhya.com/blog/2020/12/beginners-take-how-logistic-regression-is-related-to-linear-regression/#:~:text=The%20Differences%20between%20Linear%20Regression,Logistic%20regression%20provides%20discreet%20output.)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
# Read in the COVID dataset.
data = pd.read_csv("./data/synth_covid.csv")
data.head()

,blood_pressure,lung_capacity,body_temperature,has_covid
0,132.894691,6.931665,39.270112,0
1,117.128239,6.715135,37.005833,1
2,108.982006,6.580677,38.079465,0
3,112.337762,5.482720,37.662576,0
4,113.165263,6.664360,36.922810,1


In [ ]:
# Check if there are any NaN values. If so, impute them with the mean of that particular column depending on 
# if the obseration has covid or not.
# Ex: If an observation has a missing value of lung capacity and no COVID, impute the missing value with the mean
# of lung_capacity only of those observations that have COVID.

In [ ]:
# from chat in class esther
# data.loc[data["has_covid"] == 0, "blood_pressure"] = data.loc[data["has_covid"] == 0, "blood_pressure"].fillna(values_no_covid)
# from filipa
# data.loc[cond_covid_pos, "blood_pressure"] = data.loc[cond_covid_pos, "blood_pressure"].fillna(blood_covid_pos_mean)

In [3]:
data.isnull().sum()

blood_pressure      3
lung_capacity       2
body_temperature    4
has_covid           0
dtype: int64

In [ ]:
# fields that have NaN values are numeric, so can calculate mean
# if categorical code pick mode as the centrality metric

In [ ]:
# first fill in missing values for blood pressure
# Filipa mistakenly thought in class that the NaN values needed to be replaced with average for entire field
# i.e., not segregated by covid
# but she showed some nice functions so I have repeated here and commented out

# SOLUTION 1

blood_pressure_mean = data["blood_pressure"].mean()
blood_pressure_mean

In [ ]:
# sanity check that get 0 NaN values
# data["blood_pressure"].fillna(blood_pressure_mean).isna().sum()

In [ ]:
# happy with sanity check so assign

In [ ]:
# data["blood_pressure"] = data["blood_pressure"].fillna(blood_pressure_mean)

In [ ]:
# raised that average needs to be conditional

In [ ]:
# need to build condition

cond_covid_pos = data["has_covid"] == 1
cond_covid_neg = data["has_covid"] == 0

mean_blood_pressure_covid_pos = data[cond_covid_pos]["blood_pressure"].mean()
mean_blood_pressure_covid_neg = data[cond_covid_neg]["blood_pressure"].mean()

mean_blood_pressure_covid_pos, mean_blood_pressure_covid_neg

In [ ]:
#data[cond_covid_pos].isnull().sum()

In [ ]:
# this was the example shown in class but it didn't work
# warning - try using .loc
#data[cond_covid_pos]["blood_pressure"] = data[cond_covid_pos]["blood_pressure"].fillna(mean_blood_pressure_covid_pos)

In [ ]:
# it didn't make the change, can issues with the data pointing to the same place
#data[cond_covid_pos].isnull().sum()

In [ ]:
# esta's solution
# remember cond_covid_pos = data["has_covid"] == 1
#data.loc[cond_covid_pos, "blood_pressure"] = data.loc[cond_covid_pos, "blood_pressure"].fillna(mean_blood_pressure_covid_pos)

In [ ]:
data.isnull().sum()

In [ ]:
# SOLUTION ALL TOGETHER
# can create a for loop to do this ...

# list of columns want to fill missing values 
col_list = ["blood_pressure", "lung_capacity", "body_temperature"]

# condition pos/neg covid
cond_covid_pos = data["has_covid"] == 1
cond_covid_neg = data["has_covid"] == 0

for col in col_list:
    mean_col_covid_pos = data[cond_covid_pos][col].mean()
    mean_col_covid_neg = data[cond_covid_neg][col].mean()

    data.loc[cond_covid_pos, col] = data.loc[cond_covid_pos, col].fillna(mean_col_covid_pos)
    data.loc[cond_covid_neg, col] = data.loc[cond_covid_neg, col].fillna(mean_col_covid_neg)   

In [ ]:
data.isnull().sum()

In [ ]:
# nick's more programtic solution

In [6]:
# drop has covid column as want all other columns in loop
col_list_2 = data.drop("has_covid", axis=1).columns

# blank dict
means = {}

for col in col_list_2:
    # store averages for covid pos/neg and col in dict
    means[col] = {
        0: np.mean(data[data["has_covid"] == 0][col]),
        1: np.mean(data[data["has_covid"] == 1][col])        
    }


In [8]:
means["blood_pressure"]

{0: 117.87762528505783, 1: 123.07133180423557}

In [9]:
data.isnull().sum()

blood_pressure      3
lung_capacity       2
body_temperature    4
has_covid           0
dtype: int64

In [10]:
for index, row in data.iterrows():
    has_covid = row["has_covid"]
    
    for col in col_list_2:
    # specific value want to change
        value = row[col]
        if np.isnan(value):
            data.loc[index, col] = means[col][int(has_covid)]
        
    

In [11]:
data.isnull().sum()

blood_pressure      0
lung_capacity       0
body_temperature    0
has_covid           0
dtype: int64

In [ ]:
# going to drop any NaN values and come back to this if I have time

In [ ]:
data.describe()

In [ ]:
data = data.dropna()

In [ ]:
# Using Seaborn, create a histogram of the blood_pressure. Separate this by has_covid.

In [ ]:
sns.histplot(
    data = data,
    x = "blood_pressure",
    hue = "has_covid"
)

In [ ]:
# What is the distribution of the column has_covid? In other words, how many patients have COVID and don't have COVID.

In [ ]:
data["has_covid"].value_counts()

In [ ]:
636/(636 + 364)

In [ ]:
# Divide the dataset into an 80/20 training and testing split with a random_state of 42.

In [ ]:
X = data.drop(["has_covid"], axis = 1)
y = data["has_covid"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, shuffle=True, random_state = 42)

In [ ]:
# Define and train/fit a logistic regression model on your dataset.

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train)

In [ ]:
# Get the class prediction of the test set.

In [ ]:
y_pred = lr.predict(X_test)
y_pred[:10]

In [ ]:
# Get the class probabilities of the test set

In [ ]:
y_prob = lr.predict_proba(X_test)
y_prob[:10]

## Metrics
For each of these metrics (Accuracy, Recall, Precision, and F1) and using <i><b>ONLY</b></i> the NumPy library and Pandas, calculate these metrics from scratch. Then, compare this to Scikit-learn's version of these metrics by importing the necessary metric. Are your results similar or different? Why/Why not?

In [ ]:
# using np and pd to calculate  tp, tn, fp and fn
tp = np.sum(np.logical_and(y_pred == 1, y_test == 1))
tn = np.sum(np.logical_and(y_pred == 0, y_test == 0))
fp = np.sum(np.logical_and(y_pred == 1, y_test == 0))
fn = np.sum(np.logical_and(y_pred == 0, y_test == 1))
print("tp ", tp, "; tn ", tn, "; fp ", fp, "; fn ", fn, "; sum ", (tp + tn + fp + fn))

### Accuracy Score
$$Accuracy = \frac{TP + TN}{TP + TN + FP + FN}$$

In [ ]:
# using np and pd 
my_accuracy = (tp + tn) / (tp + tn + fp + fn)
print("my_accuracy ", my_accuracy)

In [ ]:
# using sklearn 
from sklearn.metrics import accuracy_score
sklearn_accuracy = accuracy_score(y_test, y_pred)
print("sklearn_accuracy ", sklearn_accuracy)

### Recall
$$Recall = \frac{TP}{TP + FN}$$

In [ ]:
my_recall = tp / (tp + fn)
print("my_recall ", my_recall)

In [ ]:
from sklearn.metrics import recall_score
sklearn_recall = recall_score(y_test, y_pred)
print("sklearn_recall ", sklearn_recall)

### Precision
$$Precision = \frac{TP}{TP + FP}$$

In [ ]:
my_precision = tp / (tp + fp)
print("my_precision ", my_precision)

In [ ]:
from sklearn.metrics import precision_score
sklearn_precision = precision_score(y_test, y_pred)
print("sklearn_precision ", sklearn_precision)

### F1-Score
$$F1= 2 * \frac{\large{precision * recall}}{\large{precision + recall}}$$

In [ ]:
my_f1 = 2 * ((my_precision * my_recall) / (my_precision + my_recall))
print("my_f1 ", my_f1)

In [ ]:
from sklearn.metrics import f1_score
sklearn_f1 = f1_score(y_test, y_pred)
print("sklearn_f1 ", sklearn_f1)

### Theoretical Questions:

1. When testing patients for COVID-19, it is extremely important to capture as many positive cases as possible to understand the prevalence of the virus within a given area. It is very dangerous to misdiagnose someone as not having the virus when in fact they do because they can spread the disease to others without knowing. On the other hand, however, if someone is healthy but diagnosed as having the virus the penalty is they will unnecessarily self-isolate for a few days, which is inconvenient but not as harmful. If this were the case, which metric would you want to maximize? Why?

It may be considered preferable to favour FN as this is dangerous to others, which would favour recall. However, the consequence of FP cannot be disregarded so you may argue that and F1 could be a better measure. 

2. Let's say that we're running a zombie task force and our job is to eliminate all zombies via flamethrower (because why not). If zombies are within the general population, let's say that these zombies are able to live freely without infecting anyone else however. We go to a brand new city to do our job and will examine every citizen to check if they are a zombie or not. Because we do not want to put an actual human through this tortorous event and would rather have the zombie live among humans, which metric would be best?

We want to minimise FP so favour precision as a metric

3. Could you give an example of when accuracy might not be the best metric to use?

If there was skew in the outcome such in this case where approximately 64% of the sample population do not have covid, so our accuracy is not much better than guessing that a participant doesn't have covid.